## Introduction

This first cell needs to be executed to set up the environment and instantiate the client used to connect to a proof assistant

In [1]:
# %pip install ollama
# %pip install colab-xterm

# !sudo apt-get update
# !sudo apt-get install pciutils lshw

# !curl -fsSL https://ollama.com/install.sh | sh
# url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/client.py"
# !wget --no-cache --backups=1 {url}
# url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/utils.py"
# !wget --no-cache --backups=1 {url}
# !pip install requests

from client import ProofAssistantClientAPI, goals_to_str

client = ProofAssistantClientAPI("http://127.0.0.1:5000")


On the server side, there is a file containing some theorems. First, let's look at them.

In [2]:
for section in client.sections():
  print(f"Theorems in section: {section}")
  for k in range(client.num_thm(section)):
    thm = client.show_thm(section, k)
    print(f"Theorem {k}, {thm['name']} description: {thm['statement']}")
  print()


Theorems in section: introduction
Theorem 0, involution_injective description: forall x y, f x = f y -> x = y

Theorems in section: logic
Theorem 0, Modus_ponens description: forall P Q : Prop, P -> (P -> Q) -> Q
Theorem 1, Modus_tollens description: forall P Q : Prop, ~Q /\ (P -> Q) -> ~P
Theorem 2, Disjunctive_syllogism description: forall P Q : Prop, (P \/ Q) -> ~P -> Q
Theorem 3, DeMorgan1 description: forall P Q : Prop, ~P \/ ~Q -> ~(P /\ Q)
Theorem 4, DeMorgan2 description: forall P Q : Prop, ~P /\ ~Q -> ~(P \/ Q)
Theorem 5, DeMorgan3 description: forall P Q : Prop, ~(P \/ Q) -> ~P /\ ~Q
Theorem 6, NotNot_LEM description: forall P : Prop, ~ ~(P \/ ~P)

Theorems in section: math
Theorem 0, unique_max description: (A : R -> Prop) (x y : R) : x is_a_max_of A -> y is_a_max_of A -> x = y
Theorem 1, inf_lt description: (A : R -> Prop) (x : R) : x is_an_inf_of A -> forall y, x < y -> exists a, A a /\ a < y
Theorem 2, le_of_le_add_eps description: (x y : R) : (forall eps, eps > 0 -> y <=

### A case study: Lelarge Theorem

We will start by proving the Lelarge Theorem in the Introduction section.
Let's review it by selecting the first theorem (index = 0) in the "introduction" section.

Before doing so, we need to examine the current context (i.e. the available lemmas, definitions, and notations) to understand it.

In [3]:
thm =  client.show_thm("introduction", 0)
print("Context:")
print("\n".join(thm['premises']))
print("\n")
print(f"Theorem {thm['name']}: {thm['statement']}",)

Context:
Variable A : Type.
Variable f : A -> A.
Hypothesis Hinv : forall x, f (f x) = x.


Theorem involution_injective: forall x y, f x = f y -> x = y


Theorem involution_injective states that an arbitrary function f, satisfying Hinv (i.e. being an involution), is one-to-one.

To prove it, we will use the following tactics (read from left to right):

* "intros x y H." introduces two variables, x and y, associated with the forall quantifier, and the hypothesis H corresponding to f x = f y.
* "rewrite <- Hinv with (x := x)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = x.
* "rewrite <- Hinv with (x := y)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = y.
* "rewrite H." rewrites the current goal using the introduced hypothesis H (i.e. f x = f y).
* "reflexivity." discharges goals of the form a = a.

### (Optionnal) A more trivial example

Theorem Modus_ponens states that if P is true and P implies Q, then Q is also true.

To prove this theorem, we will use the following tactics (read from left to right):

* "intros P Q." introduces the two propositions P and Q into the context—that is, they become available assumptions we can use.
* "intros H H0." introduces the two hypotheses: H, a proof of P, and H0, a proof of P => Q, into the context.
* "apply H0." At this point, our goal is to prove Q. Since we have a proof of P => Q (namely H0), applying it reduces our goal to proving P.
* "exact H." Now the goal is to prove P. Since we already have a proof of P in our context, we can use it directly with the exact tactic.

In [4]:
state, goals = client.start_thm("logic", 0)
print("Started theorem session:")
print(goals_to_str(goals))

Started theorem session:
Status: ongoing
Goal 1:

|-forall P Q : Prop, P -> (P -> Q) -> Q



In [5]:

state, goals = client.run_tac(state, "intros P Q.")
print(goals_to_str(goals))

Status: ongoing
Goal 1:
P, Q  : Prop
|-P -> (P -> Q) -> Q



In [6]:

state, goals = client.run_tac(state, "intros H H0.")
print(goals_to_str(goals))

Status: ongoing
Goal 1:
P, Q  : Prop
H  : P
H0  : P -> Q
|-Q



In [7]:

state, goals = client.run_tac(state, "apply H0.")
print(goals_to_str(goals))

Status: ongoing
Goal 1:
P, Q  : Prop
H  : P
H0  : P -> Q
|-P



In [8]:

state, goals = client.run_tac(state, "exact H.")
print(goals_to_str(goals))

Status: finished



## Generation

After this quick introduction, let's experiment different strategies of generation.
Ideas and some code snippets come from [Guillaume Baudart's notebook](https://github.com/LLM4Rocq/pytanque/blob/main/examples/getting_started.ipynb) and [Kaiyu Yang @ NeurIPS 2023 tutorial](https://github.com/yangky11/ml4tp-tutorial/blob/main/main.ipynb).

First strategy, let's try to generate random tactics hoping along the lines to find a correct proof.

In [9]:
from utils import generate_tactics, beam_search

generate_tactics(3)

(['left.', 'unfold not.', 'right.'],
 [0.9813818812725896, 0.5765404221171564, 0.6036429348759531])

In [17]:
section = "logic"
print(f"Theorems in section: {section}")
for k in range(client.num_thm(section)):
    thm = client.show_thm(section, k)
    print(f"Theorem {k}, {thm['name']} description: {thm['statement']}")
    print()

Theorems in section: logic
Theorem 0, Modus_ponens description: forall P Q : Prop, P -> (P -> Q) -> Q

Theorem 1, Modus_tollens description: forall P Q : Prop, ~Q /\ (P -> Q) -> ~P

Theorem 2, Disjunctive_syllogism description: forall P Q : Prop, (P \/ Q) -> ~P -> Q

Theorem 3, DeMorgan1 description: forall P Q : Prop, ~P \/ ~Q -> ~(P /\ Q)

Theorem 4, DeMorgan2 description: forall P Q : Prop, ~P /\ ~Q -> ~(P \/ Q)

Theorem 5, DeMorgan3 description: forall P Q : Prop, ~(P \/ Q) -> ~P /\ ~Q

Theorem 6, NotNot_LEM description: forall P : Prop, ~ ~(P \/ ~P)



In [14]:
section = "logic"
for k in range(client.num_thm(section)):
    proof = beam_search(client, section, k, max_depth=16, beam_size=32, timeout=10.)

    if proof:
        print(f"Proof: {" ".join(proof)}")
    else:
        print("No proof found...")

Proof: intros. apply H0. exact H.
Proof: unfold not. intros. inversion H. apply H. apply H. exact H0.
No proof found...
No proof found...
No proof found...
No proof found...
No proof found...


## Automatic theorem proving with LLMs

In the following section, we leverage a small LLM (the 7b distilled version of deepseek-r1).
We run it locally.

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])

In [ ]:
# !ollama pull deepseek-r1:7b

In [ ]:
import ollama

def get_response(prompt):
  response = ollama.chat(
        model="deepseek-r1:7b",
        messages=[
            {"role": "user", "content": prompt},
        ],
        options={"temperature":0.6}
    )
  return response["message"]["content"]

In [13]:
prompt_template = """Here is a lemma in Rocq (formerly known as Coq): 

"{statement}"

Here are some notations, definitions, and lemmas useful for understanding and proving the provided theorem:

"{premises}"

Think step by step to find a sequence of tactics in Rocq/Coq to solve this lemma. Write the final sequence of Rocq/Coq tactics between \\boxed{{}}.

Here is an example of a correct final output for another lemma:
\\boxed{unfold not.
  intros P Q H H0.
  inversion H.
  apply H1.
  apply H2.
  exact H0.}
"""

thm =  client.show_thm("introduction", 0)
statement = f"Theorem {thm['name']}: {thm['statement']}"
premises = "\n".join(thm['premises'])

prompt = prompt_template.format(statement=statement, premises=premises)
# response = get_response(prompt)
# print(response)

print(prompt)

Here is a lemma in Rocq (formerly known as Coq): 

"Theorem involution_injective: forall x y, f x = f y -> x = y"

Here is some notations, definitions, and lemmas useful to understand and prove the theorem:

"Variable A : Type.
Variable f : A -> A.
Hypothesis Hinv : forall x, f (f x) = x."

Think step by step to find a sequence of tactics in Rocq/Coq to solve this lemma. Write the final sequence of Rocq/Coq tactics between \boxed{}.



Now, let's try to find better prompts!